* https://onlinelibrary.wiley.com/doi/10.1002/sim.3561
  * https://onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1002%2Fsim.3561&file=sim_3561_sm_SupplMat.pdf

の再現

In [1]:
using Random
Random.seed!(4649373)

using Distributions
using QuadGK
using Roots
#using StatsPlots
#default(fmt=:png)

function winningrate(
        distx::ContinuousUnivariateDistribution,
        disty::ContinuousUnivariateDistribution
    )
    quadgk(y -> cdf(distx, y)*pdf(disty, y), extrema(disty)...)[1]
end

function tieshift(
        distx::ContinuousUnivariateDistribution,
        disty::ContinuousUnivariateDistribution;
        p = 1/2
    )
    find_zero(0.0) do a
        winningrate(distx + a, disty) - p
    end
end

@show distx = Exponential(1)
@show disty = Exponential(1.5)
@show winningrate(distx, disty)
println()
@show tiesh = tieshift(distx, disty)
@show mediansh = median(disty) - median(distx)
@show meansh = mean(disty) - mean(distx)
println()
@show winningrate(distx + tiesh, disty)
@show winningrate(distx + mediansh, disty)
@show winningrate(distx + meansh, disty)
println()
@show median(distx + tiesh)
@show median(distx + mediansh)
@show median(distx + meansh)
@show median(disty)
println()
@show mean(distx + tiesh)
@show mean(distx + mediansh)
@show mean(distx + meansh)
@show mean(disty)
;

distx = Exponential(1) = Exponential{Float64}(θ=1.0)
disty = Exponential(1.5) = Exponential{Float64}(θ=1.5)
winningrate(distx, disty) = 0.6000000000000003

tiesh = tieshift(distx, disty) = 0.27348233139104033
mediansh = median(disty) - median(distx) = 0.3465735902799726
meansh = mean(disty) - mean(distx) = 0.5

winningrate(distx + tiesh, disty) = 0.5
winningrate(distx + mediansh, disty) = 0.47622031634305906
winningrate(distx + meansh, disty) = 0.4299187894696709

median(distx + tiesh) = 0.9666295119509856
median(distx + mediansh) = 1.0397207708399179
median(distx + meansh) = 1.1931471805599454
median(disty) = 1.0397207708399179

mean(distx + tiesh) = 1.2734823313910404
mean(distx + mediansh) = 1.3465735902799727
mean(distx + meansh) = 1.5
mean(disty) = 1.5


In [2]:
using Distributions

@doc raw"""
    brunner_munzel_test(X, Y; p = 1/2)

この函数は数値のベクトルのデータ `X`, `Y` について, Brunner-Munzel検定関係の統計量達を計算する. 詳細は以下の通り.

函数 $H(x, y)$ と $\hat{p}$, $H^x_i$, $H^y_j$, $\bar{H}^x$, $\bar{H}^y$ を次のように定める:

```math
\begin{aligned}
&
m = \mathrm{length}(X), \quad
n = \mathrm{length}(Y), \quad
x_i = X[i], \quad
y_j = Y[j],
\\ &
\hat{p} = \frac{1}{mn}\sum_{i=1}^m \sum_{j=1}^n H(x_i, y_j),
\\ &
H(x, y) = \begin{cases} 1 & (x < y) \\ 1/2 & (x = y) \\ 0 & (x > y), \end{cases}
\\ &
H^x_i = \sum_{j=1}^n H(y_j, x_i), \quad
H^y_j = \sum_{i=1}^m H(x_i, y_j),
\\ &
\bar{H}^x = \frac{1}{m} \sum_{i=1}^m H^x_i = n - n\hat{p}, \quad
\bar{H}^y = \frac{1}{n} \sum_{j=1}^n H^y_j = m\hat{p}.
\end{aligned}
```

この函数は以下達の named tuple で返す:

```math
\begin{aligned}
&
\mathrm{phat} = 
\hat{p} = \frac{\bar{H}^y - \bar{H}^x + n}{m + n},
\\ &
\mathrm{sx2} =
\hat{\sigma}_x^2 = \frac{1}{n^2}\frac{1}{m-1}\sum_{i=1}^m (H^x_i - \bar{H}^x)^2,
\\ &
\mathrm{sy2} =
\hat{\sigma}_y^2 = \frac{1}{m^2}\frac{1}{n-1}\sum_{j=1}^n (H^y_j - \bar{H}^y)^2,
\\ &
\mathrm{sehat} = 
\widehat{\mathrm{se}} = \sqrt{\frac{\hat{\sigma}_x^2}{m} + \frac{\hat{\sigma}_y^2}{n}}, 
\\ &
\mathrm{tvalue} = t = \frac{\hat{p} - p}{\widehat{\mathrm{se}}},
\\ &
\mathrm{df} =
\nu = 
\frac
{\left(\hat{\sigma}_x^2/m + \hat{\sigma}_y^2/n\right)^2}
{
\dfrac{\left(\hat{\sigma}_x^2/m\right)^2}{m-1} +
\dfrac{\left(\hat{\sigma}_y^2/n\right)^2}{n-1}
},
\\ &
\mathrm{pvalue} =
2\mathrm{ccdf}(\mathrm{TDist}(\nu), |t|),
\\ &
\mathrm{p} = p.
\end{aligned}
```
"""
function brunner_munzel_test(X, Y; p=1/2)
    m, n = length(X), length(Y)
    phat = mean((x < y) + (x == y)/2 for x in X, y in Y)
    Hbarx = n*(1 - phat)
    Hbary = m*phat
    sx2 = 1/n^2 * 1/(m-1) * sum(x -> (sum((y < x) + (y == x)/2 for y in Y) - Hbarx)^2, X)
    sy2 = 1/m^2 * 1/(n-1) * sum(y -> (sum((x < y) + (x == y)/2 for x in X) - Hbary)^2, Y)
    sehat = √(sx2/m + sy2/n)
    tvalue = (phat - p)/sehat
    df = (sx2/m + sy2/n)^2 / ((sx2/m)^2/(m-1) + (sy2/n)^2/(n-1))
    pvalue = sehat == 0 ? 0.0 : 2ccdf(TDist(df), abs(tvalue))
    (; phat, sehat, tvalue, df, pvalue, p)
end

@doc raw"""
    pvalue_brunner_munzel_test(X, Y; p = 1/2)

この函数はBrunner-Munzel検定のP値 `pvalue` を返す.
"""
pvalue_brunner_munzel_test(X, Y; p=1/2) = brunner_munzel_test(X, Y; p).pvalue

using RCall
@rimport lawstat
X = rand(100)
Y = rand(100)
pvalue_brunner_munzel_test(X, Y), rcopy(lawstat.brunner_munzel_test(X, Y))[:p_value]

(0.10237071161776065, 0.10237071161776079)

In [3]:
X = fill(1, 100)
Y = fill(0, 100)
@show pvalue_brunner_munzel_test(X, Y), rcopy(lawstat.brunner_munzel_test(X, Y))[:p_value]
brunner_munzel_test(X, Y) |> pairs

(pvalue_brunner_munzel_test(X, Y), (rcopy(lawstat.brunner_munzel_test(X, Y)))[:p_value]) = (0.0, NaN)


pairs(::NamedTuple) with 6 entries:
  :phat   => 0.0
  :sehat  => 0.0
  :tvalue => -Inf
  :df     => NaN
  :pvalue => 0.0
  :p      => 0.5

In [4]:
X = fill(0, 100)
Y = fill(1, 100)
@show pvalue_brunner_munzel_test(X, Y), rcopy(lawstat.brunner_munzel_test(X, Y))[:p_value]
brunner_munzel_test(X, Y) |> pairs

(pvalue_brunner_munzel_test(X, Y), (rcopy(lawstat.brunner_munzel_test(X, Y)))[:p_value]) = (0.0, NaN)


pairs(::NamedTuple) with 6 entries:
  :phat   => 1.0
  :sehat  => 0.0
  :tvalue => Inf
  :df     => NaN
  :pvalue => 0.0
  :p      => 0.5

In [5]:
@doc brunner_munzel_test

```
brunner_munzel_test(X, Y; p = 1/2)
```

この函数は数値のベクトルのデータ `X`, `Y` について, Brunner-Munzel検定関係の統計量達を計算する. 詳細は以下の通り.

函数 $H(x, y)$ と $\hat{p}$, $H^x_i$, $H^y_j$, $\bar{H}^x$, $\bar{H}^y$ を次のように定める:

$$
\begin{aligned}
&
m = \mathrm{length}(X), \quad
n = \mathrm{length}(Y), \quad
x_i = X[i], \quad
y_j = Y[j],
\\ &
\hat{p} = \frac{1}{mn}\sum_{i=1}^m \sum_{j=1}^n H(x_i, y_j),
\\ &
H(x, y) = \begin{cases} 1 & (x < y) \\ 1/2 & (x = y) \\ 0 & (x > y), \end{cases}
\\ &
H^x_i = \sum_{j=1}^n H(y_j, x_i), \quad
H^y_j = \sum_{i=1}^m H(x_i, y_j),
\\ &
\bar{H}^x = \frac{1}{m} \sum_{i=1}^m H^x_i = n - n\hat{p}, \quad
\bar{H}^y = \frac{1}{n} \sum_{j=1}^n H^y_j = m\hat{p}.
\end{aligned}
$$

この函数は以下達の named tuple で返す:

$$
\begin{aligned}
&
\mathrm{phat} = 
\hat{p} = \frac{\bar{H}^y - \bar{H}^x + n}{m + n},
\\ &
\mathrm{sx2} =
\hat{\sigma}_x^2 = \frac{1}{n^2}\frac{1}{m-1}\sum_{i=1}^m (H^x_i - \bar{H}^x)^2,
\\ &
\mathrm{sy2} =
\hat{\sigma}_y^2 = \frac{1}{m^2}\frac{1}{n-1}\sum_{j=1}^n (H^y_j - \bar{H}^y)^2,
\\ &
\mathrm{sehat} = 
\widehat{\mathrm{se}} = \sqrt{\frac{\hat{\sigma}_x^2}{m} + \frac{\hat{\sigma}_y^2}{n}}, 
\\ &
\mathrm{tvalue} = t = \frac{\hat{p} - p}{\widehat{\mathrm{se}}},
\\ &
\mathrm{df} =
\nu = 
\frac
{\left(\hat{\sigma}_x^2/m + \hat{\sigma}_y^2/n\right)^2}
{
\dfrac{\left(\hat{\sigma}_x^2/m\right)^2}{m-1} +
\dfrac{\left(\hat{\sigma}_y^2/n\right)^2}{n-1}
},
\\ &
\mathrm{pvalue} =
2\mathrm{ccdf}(\mathrm{TDist}(\nu), |t|),
\\ &
\mathrm{p} = p.
\end{aligned}
$$


In [6]:
function mann_whitney_u_test(X, Y; correct=true)
    m, n = length(X), length(Y)
    phat = mean((x < y) + (x == y)/2 for x in X, y in Y)
    sehat = √((m+n+1)/(12m*n))
    zvalue = (phat - 1/2)/sehat
    correction = correct/(2m*n*sehat)
    pvalue = 2ccdf(Normal(), max(0, abs(zvalue) - correction))
    (; phat, sehat, zvalue, pvalue)
end

pvalue_mann_whitney_u_test(X, Y; correct=true) = mann_whitney_u_test(X, Y; correct).pvalue

using HypothesisTests
X = randn(100)
Y = randn(100)
pvalue_mann_whitney_u_test(X, Y), pvalue(ApproximateMannWhitneyUTest(X, Y))

(0.04990007292443201, 0.04990007292443193)

In [7]:
function student_t_test(X, Y; μ = 0.0)
    m, X̄, SX2 = length(X), mean(X), var(X)
    n, Ȳ, SY2 = length(Y), mean(Y), var(Y)
    S2 = ((m-1)*SX2 + (n-1)*SY2) / (m+n-2)
    sehat2 = S2 * (1/m + 1/n)
    tvalue = (X̄ - Ȳ - μ) / √sehat2
    df = m + n - 2
    pvalue = 2ccdf(TDist(df), abs(tvalue))
    (; pvalue, tvalue, sehat2, df)
end

pvalue_student_t_test(X, Y; μ = 0.0) = student_t_test(X, Y; μ).pvalue

using HypothesisTests
X = randn(100)
Y = randn(100)
pvalue_student_t_test(X, Y), pvalue(EqualVarianceTTest(X, Y))

(0.4389806931272048, 0.4389806931272048)

In [8]:
function welch_t_test(X, Y; μ = 0.0)
    m, X̄, SX2 = length(X), mean(X), var(X)
    n, Ȳ, SY2 = length(Y), mean(Y), var(Y)
    sehat2 = SX2/m + SY2/n
    tvalue = (X̄ - Ȳ - μ) / √sehat2
    df = sehat2^2 / ((SX2/m)^2/(m-1) + (SY2/n)^2/(n-1))
    pvalue = 2ccdf(TDist(df), abs(tvalue))
    (; pvalue, tvalue, sehat2, df)
end

pvalue_welch_t_test(X, Y; μ = 0.0) =welch_t_test(X, Y; μ).pvalue

using HypothesisTests
X = randn(100)
Y = randn(100)
pvalue_welch_t_test(X, Y), pvalue(UnequalVarianceTTest(X, Y))

(0.7219945156946428, 0.7219945156946428)

In [9]:
"""
gammadist(σ, β)

returns the gamma distribution with standard deviation `σ` and skewness `β`.
"""
gammadist(σ, β) = β == 0 ? Normal(0, σ) : Gamma(4/β^2, β*σ/2)

gam = gammadist.(1:5, 2:2:10)
std.(gam), skewness.(gam), kurtosis.(gam)

([1.0, 2.0, 3.0, 4.0, 5.0], [2.0, 4.0, 6.0, 8.0, 10.0], [6.0, 24.0, 54.0, 96.0, 150.0])

In [10]:
@doc gammadist

gammadist(σ, β)

returns the gamma distribution with standard deviation `σ` and skewness `β`.


In [11]:
"""
inversegammadist(σ, β)

returns the inverse gamma distribution with standard deviation `σ` and skewness `β`.
"""
function inversegammadist(σ, β)
    β == 0 && return Normal(0, σ)
    γ = 1/β
    α = 3 + 8γ^2 + 4γ * √(1 + 4γ^2)
    θ = σ * (α - 1) * √(α - 2)
    InverseGamma(α, θ)
end

igam = inversegammadist.(1:5, 2:2:10)
std.(igam), skewness.(igam), kurtosis.(igam)

([0.9999999999999999, 2.0, 3.0, 3.9999999999999996, 5.0], [2.0, 4.0, 6.0, 7.999999999999998, 10.0], [9.134446499564898, 72.54101966249682, NaN, NaN, NaN])

In [12]:
@doc inversegammadist

inversegammadist(σ, β)

returns the inverse gamma distribution with standard deviation `σ` and skewness `β`.


In [13]:
undefarray(T::Type, n...) = Array{T}(undef, n...)
undefarray(n...) = undefarray(Float64, n...)
ECDF(A, x) = count(≤(x), A) / length(A)

list_skewness = ((β, β) for β in 0:0.5:3)
list_skewness2 = ((β, β-0.5) for β in 1:0.5:3)
list_stdratio = reverse((1.0, 1.1, 1.25, 1.5, 2.0))

rd(x) = round(100x; digits=1)

function print_sim(;
        list_skewness = list_skewness,
        list_stdratio = list_stdratio,
        distfunc = gammadist,
        α = 0.05,
        L = 10^5,
        m = 25,
        n = 25,
        shifttype = :auto,
    )
    
    nx = length(list_skewness)
    ny = length(list_stdratio)
    # error rates
    er_wmw = undefarray(nx, ny)
    er_bm = undefarray(nx, ny)
    er_st = undefarray(nx, ny)
    er_we = undefarray(nx, ny)
    
    for (i, (β1, β2)) in enumerate(list_skewness), (j, σ) in enumerate(list_stdratio)
        distx = distfunc(σ, β1)
        disty = distfunc(1, β2)
        (; pval_wmw, pval_bm, pval_st, pval_we) = sim(distx, disty, m, n; shifttype, L)
        er_wmw[i, j] = ECDF(pval_wmw, α)
        er_bm[i, j] = ECDF(pval_bm, α)
        er_st[i, j] = ECDF(pval_st, α)
        er_we[i, j] = ECDF(pval_we, α)
    end
    
    println("Wilcoxon-Mann-Whitney:"); Base.print_matrix(stdout, rd.(er_wmw'))
    println("\n\nBrunner-Munzel:"); Base.print_matrix(stdout, rd.(er_bm'))
    println("\n\nStudent t:"); Base.print_matrix(stdout, rd.(er_st'))
    println("\n\nWelch t:"); Base.print_matrix(stdout, rd.(er_we'))
    println("\n")
end

function sim(distx, disty, m, n; shifttype=:mean, L=10^5)
    meansh = mean(disty) - mean(distx)
    mediansh = median(disty) - median(distx)
    tiesh = tieshift(distx, disty)
    if shifttype == :mean
        distx_sh = distx + meansh
    elseif shifttype == :median
        distx_sh = distx + mediansh
    elseif shifttype == :tie || shifttype == :auto
        distx_sh = distx + tiesh
    else
        distx_sh = distx
    end
    
    pval_wmw = undefarray(L)
    pval_bm = undefarray(L)
    pval_st = undefarray(L)
    pval_we = undefarray(L)
    nth = Threads.nthreads()
    Xtmp = [undefarray(m) for _ in 1:nth]
    Ytmp = [undefarray(n) for _ in 1:nth]
    Threads.@threads for i in 1:L
        tid = Threads.threadid()
        X = rand!(distx_sh, Xtmp[tid])
        Y = rand!(disty, Ytmp[tid])
        pval_wmw[i] = pvalue_mann_whitney_u_test(X, Y)
        pval_bm[i] = pvalue_brunner_munzel_test(X, Y)
        if shifttype == :auto
            @. X = X - tiesh + meansh
        end
        pval_st[i] = pvalue_student_t_test(X, Y)
        pval_we[i] = pvalue_welch_t_test(X, Y)
    end
    
    (; pval_wmw, pval_bm, pval_st, pval_we)
end

sim (generic function with 1 method)

原論文は不適切な比較をしていたので, その訂正版を以下で作成した.

In [14]:
print_sim(; m=25, n=25, list_skewness = list_skewness)

Wilcoxon-Mann-Whitney:
 5.6  5.6  5.7  5.6  5.6  5.6  5.7
 5.1  5.0  5.1  5.0  5.1  5.1  5.2
 4.8  4.8  4.9  4.8  4.9  5.0  4.9
 4.8  4.9  4.8  4.8  4.7  4.7  4.7
 4.7  4.7  4.8  4.7  4.8  4.7  4.6

Brunner-Munzel:
 5.2  5.2  5.2  5.1  5.1  5.1  5.3
 5.2  5.2  5.3  5.2  5.1  5.2  5.3
 5.3  5.3  5.3  5.2  5.3  5.3  5.2
 5.2  5.3  5.2  5.3  5.2  5.2  5.2
 5.2  5.2  5.2  5.2  5.2  5.2  5.1

Student t:
 5.2  5.3  5.4  5.7  6.2  6.4  7.1
 5.1  5.1  5.3  5.2  5.3  5.4  5.7
 5.0  5.1  5.0  5.1  5.0  5.0  4.9
 5.0  5.1  4.9  4.9  4.7  4.6  4.4
 5.0  5.0  5.0  4.9  4.8  4.6  4.3

Welch t:
 5.0  5.1  5.2  5.5  6.1  6.3  7.0
 5.0  5.0  5.2  5.1  5.2  5.2  5.5
 5.0  5.0  4.9  5.0  4.9  4.8  4.6
 5.0  5.1  4.9  4.8  4.6  4.4  4.2
 4.9  5.0  4.9  4.8  4.6  4.4  4.1



In [15]:
print_sim(; m=50, n=50, list_skewness = list_skewness)

Wilcoxon-Mann-Whitney:
 5.9  5.8  5.8  5.9  5.9  5.9  5.9
 5.4  5.2  5.3  5.3  5.4  5.4  5.3
 5.1  5.0  5.2  5.1  5.0  5.3  5.2
 5.0  5.0  4.9  5.0  5.0  5.1  5.0
 5.0  5.0  4.9  5.0  4.9  5.1  5.0

Brunner-Munzel:
 5.1  5.0  5.1  5.0  5.1  5.0  5.1
 5.2  5.1  5.0  5.0  5.2  5.2  5.0
 5.1  5.0  5.2  5.1  5.0  5.2  5.2
 5.1  5.1  5.0  5.1  5.1  5.2  5.1
 5.1  5.1  5.0  5.1  5.0  5.3  5.2

Student t:
 5.1  5.0  5.2  5.2  5.6  5.9  6.2
 5.1  5.0  5.0  5.1  5.2  5.2  5.3
 5.0  4.9  5.1  5.0  4.9  5.1  5.0
 5.1  5.0  4.9  4.9  5.0  4.9  4.8
 5.0  4.9  4.9  5.0  4.9  4.9  4.7

Welch t:
 5.1  4.9  5.1  5.1  5.5  5.8  6.1
 5.1  4.9  5.0  5.0  5.1  5.1  5.3
 5.0  4.9  5.1  5.0  4.9  5.0  4.9
 5.1  5.0  4.8  4.9  4.9  4.8  4.7
 5.0  4.9  4.9  5.0  4.8  4.8  4.6



In [16]:
print_sim(; m=25, n=100, list_skewness = list_skewness)

Wilcoxon-Mann-Whitney:
 11.0  10.8  10.9  10.8  10.9  11.0  10.8
  8.3   8.3   8.3   8.4   8.5   8.6   8.7
  6.7   6.6   6.8   6.9   6.9   7.0   7.0
  5.6   5.5   5.7   5.8   5.7   5.8   5.9
  5.0   4.9   5.0   5.0   4.8   4.9   4.7

Brunner-Munzel:
 5.2  5.1  5.1  5.1  5.0  5.2  5.1
 5.2  5.2  5.1  5.2  5.2  5.1  5.2
 5.2  5.1  5.2  5.2  5.1  5.1  5.2
 5.2  5.0  5.2  5.3  5.1  5.2  5.3
 5.3  5.2  5.3  5.3  5.1  5.2  5.0

Student t:
 18.0  18.0  18.0  18.3  18.2  18.6  18.7
 12.1  12.2  12.1  12.1  12.1  11.8  12.0
  8.7   8.4   8.7   8.5   8.2   8.0   8.2
  6.4   6.2   6.4   6.4   6.1   6.0   5.9
  5.0   5.0   5.0   5.0   4.8   4.7   4.5

Welch t:
 5.1  5.1  5.6  6.3  7.2  8.4  9.5
 5.0  5.2  5.5  6.0  6.8  7.7  8.9
 5.2  5.0  5.5  6.0  6.5  7.1  8.3
 5.0  4.9  5.5  5.9  6.5  6.9  7.7
 5.2  5.2  5.4  5.7  6.1  6.6  7.2



In [17]:
print_sim(; m=100, n=25, list_skewness = list_skewness)

Wilcoxon-Mann-Whitney:
 1.6  1.5  1.5  1.6  1.5  1.6  1.7
 2.5  2.5  2.4  2.4  2.3  2.3  2.4
 3.5  3.4  3.3  3.1  3.2  3.2  3.2
 4.2  4.2  4.1  4.1  4.0  4.0  4.2
 4.9  4.8  4.8  4.8  4.8  4.8  4.8

Brunner-Munzel:
 5.2  5.1  4.9  5.1  5.0  5.1  5.1
 5.1  5.1  5.1  5.1  5.0  5.2  5.2
 5.3  5.2  5.1  5.0  5.2  5.1  5.1
 5.1  5.2  5.2  5.1  5.1  5.2  5.3
 5.3  5.0  5.1  5.1  5.1  5.1  5.1

Student t:
 0.4  0.5  0.5  0.7  0.8  1.0  1.2
 1.3  1.4  1.4  1.5  1.6  1.8  1.9
 2.6  2.5  2.5  2.5  2.6  2.7  2.8
 3.8  3.9  3.8  3.7  3.6  3.8  3.8
 5.0  4.8  4.9  4.8  4.8  4.6  4.6

Welch t:
 5.1  5.1  4.9  5.0  4.8  4.9  4.6
 5.0  5.0  5.1  5.2  5.2  5.4  5.4
 5.1  5.1  5.2  5.4  5.8  6.0  6.3
 5.1  5.1  5.2  5.6  5.9  6.5  6.9
 5.1  5.0  5.3  5.7  6.1  6.6  7.1



In [18]:
print_sim(; m=100, n=100, list_skewness = list_skewness)

Wilcoxon-Mann-Whitney:
 5.8  6.0  5.9  6.0  6.0  6.0  6.0
 5.3  5.3  5.4  5.3  5.5  5.5  5.5
 5.2  5.1  5.0  5.0  5.1  5.1  5.1
 5.2  5.0  4.9  4.9  5.0  5.0  5.0
 5.1  5.0  5.0  5.0  5.1  5.1  4.9

Brunner-Munzel:
 5.0  5.2  5.1  5.1  5.1  5.0  5.0
 5.1  5.0  5.1  5.0  5.1  5.1  5.0
 5.2  5.0  4.9  4.9  5.0  5.0  5.0
 5.2  5.1  4.9  5.0  5.1  5.0  5.1
 5.2  5.0  5.1  5.1  5.2  5.1  5.0

Student t:
 5.0  5.2  5.2  5.3  5.4  5.5  5.7
 5.0  5.0  5.1  5.1  5.2  5.1  5.3
 5.1  4.9  4.9  5.0  5.0  5.0  4.9
 5.0  5.0  4.9  5.0  4.9  5.0  4.9
 5.1  5.0  4.9  5.0  4.9  5.0  4.9

Welch t:
 5.0  5.2  5.1  5.3  5.4  5.4  5.7
 5.0  5.0  5.0  5.1  5.2  5.1  5.2
 5.1  4.9  4.9  5.0  5.0  4.9  4.9
 5.0  5.0  4.9  5.0  4.9  4.9  4.9
 5.1  5.0  4.9  5.0  4.9  5.0  4.9



In [19]:
print_sim(; m=25, n=25, list_skewness = list_skewness2)

Wilcoxon-Mann-Whitney:
 5.5  5.4  5.4  5.2  5.2
 5.0  5.0  4.8  4.9  4.9
 4.8  4.7  5.0  4.7  4.8
 4.7  4.8  4.9  4.8  4.7
 4.8  4.9  4.9  4.8  5.0

Brunner-Munzel:
 5.2  5.2  5.2  5.1  5.2
 5.3  5.2  5.1  5.3  5.3
 5.2  5.1  5.4  5.1  5.2
 5.1  5.3  5.3  5.3  5.1
 5.4  5.3  5.3  5.2  5.3

Student t:
 5.5  5.7  6.3  6.6  7.0
 5.3  5.4  5.4  5.6  5.7
 5.1  5.1  5.2  4.8  4.9
 4.9  5.0  4.9  4.7  4.5
 5.1  5.0  4.9  4.5  4.6

Welch t:
 5.4  5.6  6.1  6.5  6.9
 5.2  5.3  5.3  5.5  5.5
 5.0  5.0  5.1  4.7  4.7
 4.8  5.0  4.8  4.5  4.3
 5.0  4.9  4.7  4.3  4.3



In [20]:
print_sim(; m=50, n=50, list_skewness = list_skewness2)

Wilcoxon-Mann-Whitney:
 5.6  5.6  5.8  5.5  5.6
 5.2  5.1  5.2  5.1  5.0
 5.1  5.0  5.0  4.9  5.1
 5.0  5.0  5.0  5.0  5.1
 5.0  5.1  5.1  5.1  5.2

Brunner-Munzel:
 4.9  5.0  5.2  5.1  5.2
 5.1  5.0  5.2  5.1  5.1
 5.2  5.1  5.1  5.0  5.2
 5.1  5.1  5.1  5.1  5.1
 5.1  5.2  5.2  5.1  5.1

Student t:
 5.1  5.4  5.8  6.1  6.3
 5.1  5.1  5.3  5.4  5.5
 5.1  4.9  5.1  5.0  5.1
 5.0  5.0  4.9  4.9  4.9
 4.9  5.1  5.0  4.8  4.8

Welch t:
 5.0  5.3  5.7  6.0  6.2
 5.0  5.1  5.2  5.3  5.4
 5.1  4.9  5.0  5.0  5.1
 5.0  5.0  4.8  4.9  4.8
 4.9  5.1  4.9  4.7  4.7



In [21]:
print_sim(; m=25, n=100, list_skewness = list_skewness2)

Wilcoxon-Mann-Whitney:
 10.5  10.1  10.0  9.9  9.4
  7.8   7.6   7.3  6.8  6.4
  6.2   5.9   5.6  5.2  4.8
  5.2   5.1   4.6  4.4  3.9
  4.5   4.3   3.9  3.6  3.4

Brunner-Munzel:
 5.2  5.1  5.1  5.2  5.2
 5.1  5.1  5.1  5.1  5.2
 5.1  5.0  5.1  5.2  5.2
 5.1  5.3  5.2  5.3  5.1
 5.2  5.3  5.1  5.0  5.1

Student t:
 17.9  18.1  18.4  18.5  18.6
 12.1  12.0  11.9  11.9  11.7
  8.6   8.4   8.3   8.2   7.9
  6.2   6.3   6.0   6.0   5.7
  4.9   5.0   4.7   4.5   4.4

Welch t:
 5.7  6.2  7.1  8.3  9.4
 5.6  6.0  7.0  7.6  8.8
 5.4  6.0  6.7  7.4  8.4
 5.3  5.9  6.4  7.1  7.8
 5.4  5.9  6.2  6.7  7.1



In [22]:
print_sim(; m=100, n=25, list_skewness = list_skewness2)

Wilcoxon-Mann-Whitney:
 1.6  1.7  1.7  1.9  2.0
 2.6  2.8  3.0  3.3  3.8
 3.6  3.9  4.3  4.7  5.1
 4.6  4.9  5.3  5.9  6.2
 5.3  5.5  6.2  6.7  7.0

Brunner-Munzel:
 5.1  5.2  5.1  5.1  5.1
 5.0  5.3  5.1  5.2  5.3
 5.1  5.2  5.1  5.2  5.2
 5.2  5.2  5.2  5.2  5.3
 5.2  5.1  5.3  5.3  5.2

Student t:
 0.6  0.7  0.8  1.0  1.3
 1.4  1.5  1.6  1.9  2.1
 2.6  2.7  2.7  2.9  2.9
 4.0  3.8  3.9  4.1  3.9
 5.1  5.0  5.0  5.0  4.9

Welch t:
 5.0  5.0  4.8  4.8  4.7
 4.9  5.2  5.1  5.2  5.4
 4.9  5.2  5.3  5.6  5.9
 5.1  5.3  5.5  6.0  6.4
 5.0  5.3  5.8  6.2  6.6



In [23]:
print_sim(; m=100, n=100, list_skewness = list_skewness2)

Wilcoxon-Mann-Whitney:
 5.7  5.6  5.7  5.6  5.6
 5.2  5.3  5.1  5.0  5.1
 5.0  5.0  5.0  5.0  5.0
 5.1  4.9  5.0  5.1  5.2
 5.0  5.1  5.1  5.3  5.2

Brunner-Munzel:
 5.0  5.0  5.0  5.0  5.1
 5.0  5.2  5.0  4.9  5.1
 5.1  5.0  5.0  5.0  5.0
 5.1  5.0  5.0  5.1  5.1
 5.0  5.1  5.1  5.3  5.0

Student t:
 5.0  5.2  5.2  5.4  5.8
 5.0  5.3  5.1  5.0  5.3
 5.0  5.1  5.1  4.9  5.0
 5.1  4.9  5.0  5.0  5.0
 5.0  5.1  5.0  5.0  4.8

Welch t:
 5.0  5.1  5.2  5.4  5.7
 5.0  5.3  5.0  5.0  5.3
 5.0  5.1  5.0  4.9  4.9
 5.1  4.9  5.0  5.0  5.0
 5.0  5.1  5.0  4.9  4.8



以下の shifttype=:mean と shifttype=:median の場合が原論文の再現になっている.

In [24]:
@time print_sim(; m=25, n=25, shifttype=:mean)

Wilcoxon-Mann-Whitney:
 5.7  6.3  8.9  13.9  22.1  33.3  46.2
 5.0  5.4  6.4   8.7  13.6  21.7  32.8
 4.7  5.0  5.3   6.2   8.1  12.5  20.1
 4.6  4.8  4.9   5.0   5.5   6.8  10.1
 4.8  4.7  4.8   4.7   4.7   4.7   4.7

Brunner-Munzel:
 5.3  5.9  8.2  12.7  19.8  29.5  40.9
 5.2  5.6  6.6   8.8  13.3  20.6  30.2
 5.1  5.4  5.7   6.7   8.6  12.8  19.5
 5.1  5.3  5.4   5.4   6.0   7.4  10.6
 5.3  5.1  5.3   5.2   5.2   5.2   5.2

Student t:
 5.3  5.3  5.4  5.7  6.1  6.5  7.3
 5.1  5.1  5.1  5.1  5.3  5.5  5.5
 5.0  5.1  5.0  5.1  5.0  4.9  4.9
 4.9  4.9  5.0  4.8  4.8  4.6  4.5
 5.2  4.9  5.0  4.8  4.8  4.7  4.4

Welch t:
 5.1  5.1  5.2  5.5  5.9  6.4  7.1
 5.0  5.0  5.0  5.0  5.2  5.3  5.4
 5.0  5.1  5.0  4.9  4.8  4.7  4.6
 4.8  4.9  5.0  4.7  4.7  4.4  4.2
 5.2  4.9  5.0  4.7  4.7  4.5  4.2

  4.722467 seconds (35.27 M allocations: 2.598 GiB, 8.61% gc time)


In [25]:
@time print_sim(; m=25, n=25, shifttype=:median)

Wilcoxon-Mann-Whitney:
 5.6  5.6  6.0  6.3  7.1  8.3  9.8
 4.9  5.1  5.2  5.6  6.0  7.1  8.5
 4.8  4.9  4.9  4.9  5.3  6.0  7.1
 4.8  4.8  4.7  4.9  4.8  5.1  5.6
 4.7  4.7  4.7  4.7  4.7  4.7  4.7

Brunner-Munzel:
 5.2  5.2  5.5  5.8  6.5  7.4  8.7
 5.1  5.3  5.3  5.7  6.1  7.0  8.2
 5.1  5.3  5.3  5.3  5.6  6.3  7.4
 5.2  5.2  5.2  5.3  5.3  5.6  6.1
 5.2  5.2  5.2  5.1  5.2  5.2  5.1

Student t:
 5.2  5.3  5.8  6.5  7.4  8.6  9.4
 5.1  5.1  5.3  5.4  5.7  6.0  6.2
 5.0  5.1  5.0  5.0  5.0  5.0  5.0
 5.0  5.1  4.9  5.0  4.8  4.6  4.7
 5.0  5.0  4.9  4.9  4.7  4.5  4.5

Welch t:
 5.0  5.1  5.5  6.1  6.9  8.0  8.7
 5.0  5.0  5.2  5.2  5.5  5.6  5.8
 5.0  5.1  4.9  4.9  4.9  4.8  4.7
 5.0  5.0  4.9  4.9  4.7  4.4  4.4
 5.0  5.0  4.9  4.8  4.6  4.3  4.3

  4.923125 seconds (35.12 M allocations: 2.555 GiB, 7.74% gc time)


In [26]:
@time print_sim(; m=25, n=25, shifttype=:tie)

Wilcoxon-Mann-Whitney:
 5.4  5.4  5.6  5.6  5.6  5.6  5.8
 5.2  4.9  5.0  5.1  5.1  5.3  5.1
 4.7  4.7  4.9  4.8  4.9  4.9  4.8
 4.9  4.8  4.8  4.8  4.7  4.6  4.7
 4.8  4.7  4.8  4.7  4.8  4.7  4.7

Brunner-Munzel:
 5.0  5.0  5.2  5.2  5.1  5.2  5.3
 5.3  5.1  5.1  5.3  5.2  5.4  5.1
 5.1  5.1  5.3  5.2  5.2  5.2  5.1
 5.3  5.3  5.2  5.2  5.2  5.1  5.2
 5.3  5.1  5.3  5.2  5.3  5.2  5.1

Student t:
 5.1  5.5  6.8  8.7  11.6  14.4  15.8
 5.2  5.1  5.7  6.6   7.4   8.6   8.8
 5.0  5.1  5.2  5.5   5.7   6.0   5.9
 5.1  5.1  4.9  5.0   4.9   4.8   4.8
 5.0  4.9  4.9  4.8   4.7   4.6   4.3

Welch t:
 4.9  5.3  6.5  8.3  10.9  13.6  14.8
 5.1  5.0  5.5  6.4   7.1   8.2   8.3
 4.9  5.0  5.1  5.3   5.5   5.8   5.6
 5.0  5.0  4.8  4.9   4.7   4.6   4.5
 4.9  4.9  4.9  4.8   4.6   4.4   4.1

  4.987917 seconds (35.32 M allocations: 2.612 GiB, 9.50% gc time)


In [27]:
@time print_sim(; m=50, n=50, shifttype=:mean)

Wilcoxon-Mann-Whitney:
 5.8  7.6  12.8  22.8  38.7  57.6  74.6
 5.3  5.9   8.1  13.5  23.4  38.7  57.5
 4.9  5.3   6.1   7.8  12.5  21.4  36.3
 5.0  5.0   5.2   5.5   6.5   9.5  16.0
 5.0  5.0   4.9   4.9   5.0   4.9   4.9

Brunner-Munzel:
 5.0  6.6  11.4  20.3  34.7  52.3  69.0
 5.1  5.7   7.8  12.9  22.0  36.0  53.4
 5.0  5.3   6.1   7.9  12.4  20.7  34.3
 5.1  5.1   5.3   5.6   6.6   9.6  15.8
 5.1  5.1   5.0   5.1   5.1   5.1   5.0

Student t:
 5.1  5.1  5.3  5.4  5.7  6.0  6.1
 5.1  5.1  5.1  5.1  5.3  5.3  5.4
 4.9  5.0  5.1  5.0  5.0  4.9  5.0
 5.0  5.1  4.9  5.0  4.9  4.7  4.6
 5.0  5.0  4.8  4.9  4.9  4.8  4.7

Welch t:
 5.0  5.0  5.2  5.3  5.6  5.9  6.0
 5.0  5.0  5.0  5.1  5.2  5.3  5.3
 4.9  5.0  5.1  5.0  5.0  4.8  4.9
 5.0  5.1  4.9  4.9  4.8  4.7  4.5
 5.0  5.0  4.8  4.8  4.9  4.8  4.7

 10.012609 seconds (35.67 M allocations: 2.709 GiB, 4.04% gc time)


In [28]:
@time print_sim(; m=50, n=50, shifttype=:median)

Wilcoxon-Mann-Whitney:
 5.8  6.0  6.4  7.4  9.0  11.2  13.9
 5.3  5.4  5.6  6.2  7.5   9.4  12.2
 4.9  5.1  5.2  5.3  5.9   7.2   9.5
 5.1  4.9  5.1  4.9  5.2   5.7   6.6
 5.1  5.0  5.0  5.0  4.9   5.0   5.1

Brunner-Munzel:
 5.0  5.2  5.6  6.4  7.7  9.5  11.8
 5.1  5.2  5.4  5.9  7.1  8.8  11.2
 5.0  5.2  5.3  5.3  5.9  7.2   9.3
 5.2  5.0  5.2  5.1  5.3  5.8   6.7
 5.2  5.1  5.1  5.1  5.1  5.1   5.2

Student t:
 5.1  5.7  7.0  9.7  13.1  16.7  20.1
 5.1  5.3  5.7  6.8   7.8   8.8   9.9
 5.0  5.1  5.2  5.4   5.7   6.0   6.2
 5.1  5.0  5.1  4.9   5.0   5.1   5.1
 5.0  5.0  5.0  4.9   4.9   4.7   4.8

Welch t:
 5.0  5.5  6.8  9.5  12.8  16.4  19.6
 5.1  5.2  5.7  6.7   7.7   8.6   9.7
 5.0  5.1  5.2  5.3   5.6   6.0   6.1
 5.1  4.9  5.1  4.9   5.0   5.1   5.0
 5.0  5.0  5.0  4.9   4.9   4.7   4.7

 10.040508 seconds (35.50 M allocations: 2.661 GiB, 4.62% gc time)


In [29]:
@time print_sim(; m=50, n=50, shifttype=:tie)

Wilcoxon-Mann-Whitney:
 5.9  5.8  5.9  5.9  6.0  6.0  5.9
 5.2  5.3  5.3  5.3  5.3  5.4  5.4
 5.1  5.0  5.1  5.1  5.1  5.1  5.2
 4.9  4.8  4.9  5.0  5.0  5.1  5.0
 4.8  5.1  5.0  4.9  5.0  4.9  4.9

Brunner-Munzel:
 5.1  5.1  5.1  5.0  5.1  5.1  5.0
 5.0  5.0  5.1  5.1  5.1  5.2  5.1
 5.1  5.0  5.2  5.1  5.2  5.1  5.1
 5.1  4.9  5.0  5.2  5.2  5.2  5.1
 5.0  5.3  5.2  5.1  5.1  5.1  5.1

Student t:
 5.2  6.1  9.7  15.8  23.6  30.8  34.7
 4.9  5.5  6.9   9.3  12.5  15.4  16.5
 5.1  5.1  5.6   6.5   7.3   8.1   8.6
 5.0  4.9  5.1   5.2   5.4   5.6   5.5
 4.9  5.1  5.1   4.9   4.9   4.8   4.7

Welch t:
 5.1  6.0  9.5  15.5  23.2  30.3  34.2
 4.8  5.5  6.9   9.2  12.4  15.2  16.3
 5.0  5.1  5.6   6.4   7.2   8.0   8.5
 5.0  4.9  5.1   5.2   5.4   5.5   5.4
 4.9  5.1  5.1   4.9   4.9   4.8   4.6

 10.225169 seconds (35.82 M allocations: 2.751 GiB, 4.52% gc time)


In [30]:
@time print_sim(; m=25, n=100, shifttype=:mean)

Wilcoxon-Mann-Whitney:
 10.6  11.9  16.6  24.1  35.7  49.3  62.9
  8.3   8.9  11.1  15.4  23.7  34.9  48.9
  6.8   7.0   7.8   9.6  13.7  21.4  33.0
  5.6   5.7   5.9   6.5   7.7  10.7  16.8
  4.9   5.0   4.9   4.9   4.8   4.8   4.9

Brunner-Munzel:
 5.0  5.8  8.6  13.1  20.6  30.6  42.5
 5.1  5.6  6.8   9.5  14.5  21.6  32.1
 5.3  5.3  5.9   6.9   9.2  13.8  21.2
 5.2  5.2  5.3   5.6   6.2   8.0  11.3
 5.1  5.3  5.2   5.2   5.2   5.1   5.2

Student t:
 17.9  18.1  18.4  18.3  18.4  18.6  18.9
 12.2  12.2  12.2  12.0  12.2  12.1  11.9
  8.7   8.7   8.7   8.4   8.2   8.2   8.1
  6.4   6.4   6.3   6.2   6.1   6.1   5.9
  4.9   5.1   5.0   4.9   4.8   4.7   4.7

Welch t:
 4.9  5.1  5.7  6.3  7.1  8.3  9.6
 5.0  5.2  5.6  6.1  6.9  7.8  8.8
 5.1  5.2  5.6  5.9  6.5  7.4  8.2
 5.0  5.2  5.3  5.8  6.3  7.0  7.7
 4.9  5.2  5.3  5.8  6.2  6.7  7.2

 10.351226 seconds (35.84 M allocations: 2.758 GiB, 4.56% gc time)


In [31]:
@time print_sim(; m=25, n=100, shifttype=:median)

Wilcoxon-Mann-Whitney:
 10.7  10.9  11.6  12.3  13.8  15.6  18.0
  8.4   8.4   8.7   9.5  10.8  12.5  15.3
  6.7   6.8   7.1   7.3   8.2   9.6  11.9
  5.6   5.6   5.8   5.9   6.2   7.1   8.3
  5.0   4.8   4.9   4.8   5.0   5.0   4.8

Brunner-Munzel:
 5.1  5.2  5.5  5.8  6.5  7.2  8.6
 5.3  5.2  5.3  5.7  6.2  6.9  8.3
 5.2  5.2  5.3  5.4  5.8  6.3  7.5
 5.1  5.2  5.2  5.3  5.4  5.8  6.1
 5.3  5.1  5.1  5.1  5.3  5.2  5.1

Student t:
 18.0  18.6  20.6  23.1  26.7  29.9  32.6
 12.3  12.3  12.9  14.0  14.9  16.0  17.0
  8.6   8.7   8.8   9.1   9.3   9.6   9.8
  6.5   6.4   6.4   6.5   6.5   6.5   6.3
  5.0   4.9   5.0   4.9   4.9   4.9   4.7

Welch t:
 4.9  5.2  5.5  5.9  6.7  7.2  7.7
 5.1  5.0  5.0  5.2  5.3  5.3  5.2
 5.0  5.1  5.2  5.1  5.1  5.0  5.2
 5.0  5.0  5.3  5.3  5.6  5.8  6.2
 5.1  5.1  5.4  5.6  6.3  6.8  7.3

 10.531491 seconds (35.64 M allocations: 2.701 GiB, 3.89% gc time)


In [32]:
@time print_sim(; m=25, n=100, shifttype=:tie)

Wilcoxon-Mann-Whitney:
 10.8  10.8  10.9  10.8  11.1  10.9  10.8
  8.3   8.2   8.4   8.6   8.5   8.6   8.6
  6.6   6.7   6.6   6.9   7.0   7.1   7.0
  5.8   5.5   5.7   5.5   5.7   5.8   5.8
  4.9   4.9   4.8   5.0   4.9   4.9   4.9

Brunner-Munzel:
 5.3  5.1  5.1  5.1  5.3  5.2  5.2
 5.2  5.1  5.2  5.2  5.1  5.1  5.1
 5.0  5.1  5.0  5.3  5.3  5.2  5.2
 5.3  5.1  5.2  5.0  5.1  5.1  5.2
 5.2  5.2  5.1  5.3  5.2  5.2  5.2

Student t:
 18.2  19.3  23.2  29.3  35.7  41.3  44.1
 12.2  12.5  14.1  16.7  19.4  21.9  22.8
  8.5   8.7   9.0  10.2  11.0  11.7  11.9
  6.6   6.3   6.5   6.6   6.6   6.9   6.7
  5.1   5.0   4.9   4.9   4.8   4.7   4.6

Welch t:
 5.0  5.3  6.4  8.3  10.4  12.5  13.6
 5.1  5.0  5.4  6.0   6.5   7.0   6.9
 4.9  5.0  4.9  5.2   5.2   5.1   5.2
 5.2  4.9  5.1  5.1   5.1   5.3   5.6
 5.0  5.1  5.2  5.8   6.1   6.7   7.2

 10.359445 seconds (35.85 M allocations: 2.761 GiB, 4.52% gc time)


In [33]:
@time print_sim(; m=100, n=25, shifttype=:mean)

Wilcoxon-Mann-Whitney:
 1.6  2.3  5.3  12.1  26.1  48.2  70.9
 2.5  2.7  4.1   7.2  13.9  27.4  47.6
 3.5  3.6  3.9   5.0   7.6  13.4  25.2
 4.2  4.2  4.3   4.4   4.8   6.4  10.3
 5.0  5.0  4.8   5.0   4.8   4.9   4.7

Brunner-Munzel:
 5.0  6.8  13.3  26.5  47.6  71.5  88.3
 5.0  5.7   8.1  13.8  25.6  45.7  68.9
 5.3  5.4   6.0   7.8  12.3  22.1  40.5
 5.2  5.2   5.4   5.7   6.4   9.1  15.9
 5.3  5.2   5.1   5.3   5.1   5.1   5.0

Student t:
 0.5  0.5  0.6  0.7  0.9  1.0  1.3
 1.3  1.4  1.4  1.5  1.6  1.7  2.0
 2.6  2.6  2.5  2.6  2.7  2.7  2.8
 3.9  3.8  3.9  3.8  3.7  3.7  3.7
 5.1  5.1  5.0  4.9  4.9  4.8  4.6

Welch t:
 5.0  5.0  5.0  4.9  4.8  4.8  4.7
 4.9  5.0  5.0  5.2  5.4  5.3  5.4
 5.1  5.0  5.1  5.5  5.8  6.0  6.2
 5.1  5.1  5.3  5.7  5.9  6.4  6.8
 5.1  5.2  5.4  5.8  6.1  6.7  7.3

 10.976370 seconds (35.26 M allocations: 2.595 GiB, 4.27% gc time)


In [34]:
@time print_sim(; m=100, n=25, shifttype=:median)

Wilcoxon-Mann-Whitney:
 1.6  1.6  1.7  2.1  2.9  4.0  5.6
 2.4  2.6  2.6  2.8  3.3  4.2  5.9
 3.3  3.4  3.4  3.5  3.6  4.1  5.2
 4.2  4.1  4.2  4.1  4.1  4.1  4.5
 4.8  4.8  4.8  5.1  4.9  4.9  4.9

Brunner-Munzel:
 5.2  5.4  5.7  6.6  8.4  11.3  14.6
 5.1  5.4  5.3  5.9  7.2   8.9  12.4
 5.1  5.1  5.2  5.4  5.9   6.9   9.0
 5.2  5.1  5.3  5.2  5.3   5.6   6.5
 5.2  5.1  5.1  5.3  5.1   5.2   5.1

Student t:
 0.4  0.5  0.8  1.2  1.8  2.6  3.5
 1.3  1.5  1.5  1.8  2.1  2.4  2.5
 2.5  2.5  2.6  2.6  2.6  2.7  2.5
 3.8  3.7  3.7  3.7  3.7  3.5  3.5
 5.0  5.0  5.0  5.0  4.8  4.7  4.7

Welch t:
 5.1  6.2  9.4  14.9  21.6  28.1  34.1
 4.8  5.7  7.1   9.6  12.7  16.0  18.4
 4.9  5.3  6.1   7.3   9.0  10.6  12.2
 5.1  5.1  5.6   6.2   7.0   8.1   8.9
 5.0  5.1  5.4   5.8   6.2   6.7   7.4

 10.827569 seconds (35.14 M allocations: 2.559 GiB, 3.85% gc time)


In [35]:
@time print_sim(; m=100, n=25, shifttype=:tie)

Wilcoxon-Mann-Whitney:
 1.5  1.5  1.6  1.5  1.5  1.7  1.6
 2.5  2.4  2.4  2.4  2.3  2.4  2.3
 3.4  3.4  3.5  3.3  3.3  3.2  3.3
 4.2  4.3  4.2  4.2  4.1  4.0  4.1
 4.9  5.0  4.8  5.0  4.9  5.0  4.8

Brunner-Munzel:
 5.2  5.0  5.1  5.2  5.1  5.2  5.1
 5.1  5.2  5.1  5.2  5.1  5.2  5.1
 5.2  5.1  5.3  5.2  5.1  5.1  5.2
 5.2  5.3  5.2  5.1  5.1  5.2  5.3
 5.1  5.2  5.2  5.3  5.1  5.2  5.1

Student t:
 0.5  0.7  1.4  3.0  5.8  9.2  11.1
 1.4  1.5  2.0  2.9  4.1  5.4   5.6
 2.6  2.6  2.9  3.0  3.5  3.8   3.8
 3.8  3.9  3.8  3.8  3.8  3.8   3.7
 4.9  5.1  5.0  4.9  4.8  4.8   4.6

Welch t:
 5.0  7.1  13.5  23.7  34.9  43.8  48.8
 5.0  6.1   8.9  13.6  18.8  24.0  26.6
 5.1  5.6   7.0   8.8  11.6  14.2  16.0
 5.0  5.3   5.8   6.8   7.9   9.4  10.3
 5.0  5.2   5.3   5.9   6.1   6.8   7.3

 11.005899 seconds (35.47 M allocations: 2.653 GiB, 4.83% gc time)


In [36]:
@time print_sim(; m=100, n=100, shifttype=:mean)

Wilcoxon-Mann-Whitney:
 5.7  9.0  19.5  39.1  64.3  84.9  95.5
 5.2  6.6  11.3  21.8  40.7  64.8  85.0
 5.0  5.6   7.1  11.0  19.9  37.7  61.3
 5.0  5.1   5.4   6.1   8.2  14.3  27.4
 5.0  5.0   5.0   5.1   4.9   4.9   5.0

Brunner-Munzel:
 4.9  7.8  17.5  35.8  60.0  81.4  93.7
 4.9  6.3  10.8  20.8  38.7  61.9  82.3
 5.0  5.6   7.1  10.9  19.5  36.5  59.0
 5.1  5.1   5.4   6.2   8.3  14.3  26.8
 5.0  5.1   5.0   5.2   4.9   4.9   5.1

Student t:
 5.0  5.0  5.1  5.3  5.3  5.5  5.6
 4.9  5.0  5.1  5.0  5.1  5.2  5.3
 5.0  5.0  5.0  5.0  5.1  5.1  5.0
 5.1  5.1  5.0  5.0  4.9  5.0  4.9
 5.0  5.1  5.0  5.0  4.9  4.9  4.8

Welch t:
 5.0  5.0  5.1  5.3  5.2  5.5  5.5
 4.9  5.0  5.0  5.0  5.1  5.2  5.3
 5.0  5.0  5.0  5.0  5.1  5.0  5.0
 5.1  5.1  5.0  5.0  4.9  5.0  4.8
 5.0  5.1  5.0  4.9  4.9  4.9  4.8

 31.177398 seconds (36.07 M allocations: 2.822 GiB, 1.36% gc time)


In [37]:
@time print_sim(; m=100, n=100, shifttype=:median)

Wilcoxon-Mann-Whitney:
 5.8  6.3  6.8  9.0  12.0  16.2  22.0
 5.4  5.6  6.0  7.1   9.6  13.3  19.0
 5.0  5.2  5.3  5.9   7.0   9.4  13.5
 4.8  4.9  5.1  5.2   5.5   6.2   8.5
 5.0  4.9  5.1  4.9   5.0   5.0   5.1

Brunner-Munzel:
 5.0  5.4  5.8  7.7  10.3  13.9  19.0
 5.1  5.3  5.7  6.7   9.0  12.4  17.5
 5.0  5.1  5.3  5.9   6.9   9.2  13.0
 4.9  5.0  5.1  5.2   5.6   6.3   8.5
 5.1  4.9  5.1  5.0   5.1   5.1   5.2

Student t:
 5.0  6.3  10.3  16.8  25.7  34.2  41.4
 5.1  5.6   7.0   9.1  12.1  15.3  17.8
 4.9  5.3   5.6   6.3   7.2   7.9   8.7
 4.8  5.0   5.1   5.3   5.3   5.3   5.5
 5.0  4.9   5.1   5.0   5.0   4.9   5.0

Welch t:
 5.0  6.3  10.2  16.7  25.5  34.0  41.1
 5.0  5.6   6.9   9.0  12.1  15.2  17.7
 4.9  5.3   5.6   6.2   7.1   7.9   8.7
 4.8  5.0   5.1   5.3   5.3   5.3   5.4
 5.0  4.9   5.1   5.0   5.0   4.9   5.0

 30.901995 seconds (36.10 M allocations: 2.833 GiB, 1.64% gc time)


In [38]:
@time print_sim(; m=100, n=100, shifttype=:tie)

Wilcoxon-Mann-Whitney:
 5.9  5.7  5.9  6.0  6.0  5.9  6.0
 5.3  5.3  5.2  5.4  5.4  5.5  5.4
 5.2  5.1  5.1  5.0  5.2  5.1  5.1
 5.0  5.1  4.9  5.0  4.9  5.0  5.0
 4.9  4.9  4.9  5.0  5.1  4.9  4.9

Brunner-Munzel:
 5.0  4.9  5.0  5.1  5.0  5.0  5.1
 5.1  5.0  5.0  5.1  5.1  5.1  5.0
 5.1  5.1  5.0  5.0  5.1  5.0  5.0
 5.0  5.1  5.0  5.0  5.0  5.0  5.0
 5.0  5.0  5.0  5.0  5.1  5.0  5.0

Student t:
 5.0  7.7  16.5  30.8  46.6  59.6  65.6
 5.1  6.0   9.6  15.5  22.6  28.7  31.3
 5.0  5.4   6.5   8.2  10.7  12.8  13.4
 5.0  5.1   5.2   5.7   6.1   6.5   6.5
 4.9  5.0   5.0   5.0   5.0   4.9   4.9

Welch t:
 5.0  7.6  16.4  30.7  46.3  59.4  65.4
 5.1  6.0   9.5  15.4  22.5  28.6  31.2
 5.0  5.4   6.5   8.2  10.7  12.8  13.4
 5.0  5.1   5.1   5.6   6.1   6.5   6.5
 4.9  5.0   5.0   4.9   5.0   4.9   4.8

 31.356783 seconds (36.51 M allocations: 2.947 GiB, 1.78% gc time)


In [39]:
@time print_sim(; m=25, n=25, shifttype=:mean, list_skewness=list_skewness2)

Wilcoxon-Mann-Whitney:
 8.8  13.3  20.9  31.5  44.0
 6.6   8.8  12.2  18.3  28.3
 5.6   6.3   7.1   8.5  12.4
 5.0   5.2   5.2   4.9   4.8
 5.0   4.9   4.7   5.1   6.6

Brunner-Munzel:
 8.2  12.4  19.4  28.8  39.8
 6.8   9.0  12.4  18.2  27.2
 6.0   6.8   7.6   9.1  13.0
 5.5   5.6   5.7   5.3   5.2
 5.5   5.3   5.1   5.4   6.9

Student t:
 5.5  5.7  6.1  6.6  7.2
 5.2  5.3  5.4  5.5  5.7
 5.1  5.0  4.9  4.9  5.0
 5.0  4.9  5.0  4.7  4.5
 5.1  4.9  4.7  4.6  4.5

Welch t:
 5.3  5.6  6.0  6.5  7.0
 5.1  5.2  5.3  5.4  5.5
 5.0  4.9  4.8  4.8  4.8
 4.9  4.8  4.9  4.5  4.3
 5.1  4.9  4.6  4.5  4.3

  3.738444 seconds (25.30 M allocations: 1.884 GiB, 7.66% gc time, 0.43% compilation time)


In [40]:
@time print_sim(; m=25, n=25, shifttype=:median, list_skewness=list_skewness2)

Wilcoxon-Mann-Whitney:
 5.4  5.4  5.6  5.7  6.1
 5.0  4.9  4.9  4.9  5.3
 4.9  5.0  5.2  5.7  7.4
 4.8  5.1  5.6  6.5  8.6
 5.1  5.3  5.9  7.2  9.5

Brunner-Munzel:
 5.1  5.2  5.3  5.5  5.9
 5.3  5.2  5.3  5.3  5.9
 5.4  5.4  5.7  6.4  8.2
 5.4  5.7  6.1  7.1  9.1
 5.6  5.8  6.4  7.7  9.8

Student t:
 7.0  8.0  9.3  10.3  10.7
 6.4  6.8  7.3   7.3   7.1
 5.9  6.2  6.2   5.9   5.6
 5.6  5.8  5.7   5.2   4.9
 5.7  5.6  5.3   5.0   4.7

Welch t:
 6.7  7.6  8.8  9.7  10.0
 6.2  6.5  7.0  6.9   6.7
 5.8  6.1  6.0  5.6   5.3
 5.5  5.7  5.6  5.0   4.7
 5.7  5.5  5.2  4.8   4.5

  3.488322 seconds (25.47 M allocations: 1.933 GiB, 7.42% gc time)


In [41]:
@time print_sim(; m=25, n=25, shifttype=:tie, list_skewness=list_skewness2)

Wilcoxon-Mann-Whitney:
 5.4  5.4  5.3  5.4  5.2
 5.0  4.9  4.9  4.9  4.8
 4.9  4.8  4.9  4.7  4.8
 4.6  4.7  4.8  4.8  4.8
 4.8  4.8  4.7  4.9  5.0

Brunner-Munzel:
 5.1  5.1  5.2  5.2  5.1
 5.2  5.2  5.2  5.2  5.2
 5.3  5.2  5.3  5.2  5.2
 5.1  5.2  5.3  5.3  5.2
 5.3  5.2  5.1  5.2  5.3

Student t:
 6.8  8.6  11.0  12.8  13.1
 5.9  6.4   7.0   6.8   6.3
 5.4  5.5   5.3   5.0   4.8
 5.1  5.1   4.9   4.7   4.7
 5.0  4.9   4.8   4.9   5.0

Welch t:
 6.5  8.2  10.4  12.1  12.3
 5.8  6.2   6.7   6.4   6.0
 5.3  5.4   5.2   4.8   4.6
 5.0  5.0   4.8   4.5   4.4
 5.0  4.8   4.7   4.7   4.8

  3.629641 seconds (25.31 M allocations: 1.888 GiB, 9.23% gc time)


In [42]:
@time print_sim(; m=50, n=50, shifttype=:mean, list_skewness=list_skewness2)

Wilcoxon-Mann-Whitney:
 12.9  22.2  36.8  55.2  72.4
  8.8  13.0  20.2  32.5  49.9
  6.8   7.9   9.7  13.0  21.6
  5.7   6.1   5.7   5.3   5.2
  5.4   5.1   4.9   5.5   8.4

Brunner-Munzel:
 11.6  20.2  33.7  51.0  67.7
  8.6  12.6  19.6  31.2  47.5
  6.9   8.0   9.7  13.1  21.5
  5.8   6.1   5.8   5.3   5.2
  5.6   5.2   5.0   5.5   8.2

Student t:
 5.3  5.5  5.7  5.9  6.3
 5.1  5.1  5.4  5.3  5.5
 5.1  4.9  5.1  5.0  5.1
 4.9  5.1  5.0  4.8  4.9
 5.0  5.0  4.8  4.7  4.8

Welch t:
 5.2  5.4  5.6  5.8  6.2
 5.1  5.1  5.3  5.2  5.5
 5.0  4.9  5.0  4.9  5.0
 4.9  5.1  5.0  4.8  4.8
 5.0  4.9  4.8  4.7  4.7

  7.780636 seconds (25.62 M allocations: 1.975 GiB, 4.19% gc time)


In [43]:
@time print_sim(; m=50, n=50, shifttype=:median, list_skewness=list_skewness2)

Wilcoxon-Mann-Whitney:
 5.8  5.8  5.8   6.4   7.1
 5.4  5.2  5.1   5.3   6.3
 5.3  5.3  5.8   7.2  10.5
 5.4  5.7  6.7   9.1  12.9
 5.6  6.0  7.5  10.3  14.6

Brunner-Munzel:
 5.2  5.1  5.2   5.8   6.4
 5.3  5.1  5.1   5.3   6.5
 5.4  5.5  6.0   7.5  10.7
 5.6  5.9  6.9   9.3  12.9
 5.8  6.2  7.7  10.3  14.2

Student t:
 10.4  13.9  18.4  21.5  23.4
  8.7  10.4  11.7  12.5  12.6
  7.6   8.2   8.5   8.5   8.0
  7.0   7.1   7.0   6.6   5.7
  6.7   6.4   6.0   5.7   5.0

Welch t:
 10.2  13.7  18.0  21.2  23.0
  8.6  10.3  11.6  12.3  12.4
  7.6   8.2   8.5   8.4   7.9
  7.0   7.0   7.0   6.6   5.7
  6.7   6.3   6.0   5.7   5.0

  7.522633 seconds (26.09 M allocations: 2.110 GiB, 4.85% gc time)


In [44]:
@time print_sim(; m=50, n=50, shifttype=:tie, list_skewness=list_skewness2)

Wilcoxon-Mann-Whitney:
 5.7  5.5  5.6  5.6  5.4
 5.2  5.2  5.1  5.1  5.0
 5.1  5.0  4.9  5.0  5.0
 5.0  4.7  5.0  5.0  5.2
 5.0  4.9  5.1  5.2  5.2

Brunner-Munzel:
 5.0  4.9  5.0  5.1  5.0
 5.0  5.1  5.1  5.2  5.1
 5.1  5.2  5.0  5.2  5.1
 5.1  4.9  5.1  5.1  5.2
 5.1  5.0  5.2  5.2  5.1

Student t:
 10.2  15.7  22.1  27.4  29.1
  7.4   9.4  10.8  11.3  10.4
  6.2   6.5   6.3   6.1   5.5
  5.5   5.4   5.1   4.8   4.8
  5.2   4.9   5.0   5.3   5.7

Welch t:
 10.0  15.4  21.7  27.0  28.7
  7.4   9.3  10.7  11.1  10.2
  6.1   6.5   6.2   6.1   5.4
  5.5   5.4   5.0   4.7   4.7
  5.2   4.9   5.0   5.2   5.6

  7.566034 seconds (25.75 M allocations: 2.011 GiB, 4.98% gc time)


In [45]:
@time print_sim(; m=25, n=100, shifttype=:mean, list_skewness=list_skewness2)

Wilcoxon-Mann-Whitney:
 16.4  23.5  34.3  47.3  61.2
 11.0  14.5  20.6  30.6  43.4
  7.9   8.8  10.2  13.3  20.6
  6.1   5.8   5.5   4.7   3.9
  5.0   4.5   3.9   3.8   5.0

Brunner-Munzel:
 8.9  13.5  20.7  30.6  41.9
 7.5   9.7  13.9  20.3  29.7
 6.5   7.3   8.3  10.6  15.3
 5.9   5.8   5.8   5.5   5.1
 5.6   5.4   5.1   5.5   7.9

Student t:
 18.3  18.2  18.4  18.6  18.7
 12.2  12.0  12.0  12.0  12.0
  8.5   8.5   8.2   8.0   7.8
  6.4   6.0   6.0   5.9   5.8
  5.0   4.8   4.7   4.6   4.5

Welch t:
 5.6  6.3  7.3  8.2  9.7
 5.5  6.0  6.9  7.8  8.8
 5.5  6.0  6.6  7.5  8.3
 5.4  5.6  6.5  7.0  7.9
 5.4  5.7  6.2  6.8  7.4

  7.740045 seconds (25.72 M allocations: 2.005 GiB, 3.95% gc time)


In [46]:
@time print_sim(; m=25, n=100, shifttype=:median, list_skewness=list_skewness2)

Wilcoxon-Mann-Whitney:
 10.5  10.3  10.5  10.9  11.2
  8.0   7.7   7.3   6.8   6.8
  6.4   6.1   5.9   6.1   7.7
  5.4   5.4   5.5   6.5   8.5
  4.9   4.8   5.3   6.4   9.0

Brunner-Munzel:
 5.1  5.1  5.3  5.6   5.8
 5.2  5.2  5.2  5.3   6.2
 5.3  5.4  5.6  6.7   9.6
 5.4  5.8  6.5  8.4  12.3
 5.6  5.9  7.1  9.5  14.3

Student t:
 24.1  27.5  31.1  34.4  35.3
 16.0  17.7  18.6  19.4  19.2
 11.0  11.7  11.9  11.7  11.4
  8.2   8.3   8.1   7.8   7.2
  6.2   6.2   5.9   5.5   5.1

Welch t:
 6.7  7.4  8.5  9.2  9.0
 6.2  6.4  6.2  6.3  5.7
 5.7  5.6  5.5  5.0  5.0
 5.6  5.4  5.1  5.1  5.3
 5.4  5.1  5.1  5.2  5.9

  8.091287 seconds (25.93 M allocations: 2.064 GiB, 3.86% gc time)


In [47]:
@time print_sim(; m=25, n=100, shifttype=:tie, list_skewness=list_skewness2)

Wilcoxon-Mann-Whitney:
 10.5  9.9  10.0  9.9  9.2
  8.0  7.6   7.4  6.9  6.3
  6.4  6.0   5.8  5.3  4.8
  5.2  5.0   4.6  4.3  3.9
  4.5  4.3   3.9  3.6  3.4

Brunner-Munzel:
 5.1  5.1  5.2  5.2  5.2
 5.1  5.1  5.3  5.1  5.2
 5.2  5.2  5.4  5.3  5.1
 5.1  5.1  5.2  5.1  5.1
 5.2  5.2  5.2  5.1  5.2

Student t:
 23.6  29.1  34.6  38.5  39.9
 14.8  16.5  17.7  18.0  17.3
  9.7   9.9  10.0   9.5   8.7
  6.8   6.7   6.3   6.0   5.8
  5.1   4.9   4.7   4.5   4.8

Welch t:
 6.5  7.9  10.0  11.2  11.2
 5.6  6.0   6.0   5.7   5.3
 5.3  5.2   5.1   5.3   5.7
 5.1  5.1   5.3   6.4   7.7
 5.0  5.3   6.3   7.9  10.1

  7.653873 seconds (25.70 M allocations: 1.997 GiB, 4.72% gc time)


In [48]:
@time print_sim(; m=100, n=25, shifttype=:mean, list_skewness=list_skewness2)

Wilcoxon-Mann-Whitney:
 5.8  12.3  25.0  44.9  67.8
 4.9   7.8  12.8  22.7  39.7
 4.8   5.8   7.1   9.3  14.6
 5.1   5.4   5.5   5.8   6.2
 5.5   5.6   6.2   7.5  10.4

Brunner-Munzel:
 13.7  25.6  44.8  67.9  86.1
  8.8  13.2  21.4  36.6  58.7
  6.9   7.9   9.5  12.6  21.0
  5.9   6.1   5.8   5.6   5.4
  5.5   5.3   5.3   5.5   6.7

Student t:
 0.6  0.6  0.8  1.0  1.2
 1.4  1.6  1.7  1.9  1.9
 2.6  2.6  2.7  2.9  2.8
 3.9  4.0  3.9  4.0  4.0
 5.0  4.9  5.0  5.0  4.9

Welch t:
 4.9  4.9  4.9  4.7  4.8
 5.0  5.2  5.0  5.3  5.1
 5.1  5.1  5.3  5.6  6.0
 5.1  5.4  5.5  5.8  6.5
 5.0  5.2  5.7  6.2  6.6

  8.123564 seconds (25.32 M allocations: 1.892 GiB, 3.42% gc time)


In [49]:
@time print_sim(; m=100, n=25, shifttype=:median, list_skewness=list_skewness2)

Wilcoxon-Mann-Whitney:
 1.7  1.8  1.8   2.3   2.6
 2.8  2.9  3.2   3.5   5.5
 4.0  4.5  5.2   7.2  11.1
 5.1  5.7  7.1   9.8  13.9
 5.8  6.7  8.6  11.5  15.9

Brunner-Munzel:
 5.1  5.3  5.4  6.1   6.8
 5.3  5.2  5.3  5.2   6.2
 5.4  5.6  5.7  6.7   8.8
 5.6  5.8  6.3  7.5   9.6
 5.6  5.8  6.8  8.0  10.2

Student t:
 1.6  2.6  3.6  4.8  5.4
 2.9  3.5  3.9  4.0  3.8
 4.1  4.4  4.3  4.3  3.8
 5.2  5.2  5.0  4.7  4.2
 6.3  6.1  6.0  5.3  4.9

Welch t:
 13.6  20.7  27.3  33.4  37.1
 10.5  13.9  16.9  19.4  21.1
  8.9  10.6  12.1  13.4  14.1
  7.7   8.9   9.8  10.1  10.3
  7.1   7.8   8.5   8.4   8.5

  8.402662 seconds (25.81 M allocations: 2.030 GiB, 4.25% gc time)


In [50]:
@time print_sim(; m=100, n=25, shifttype=:tie, list_skewness=list_skewness2)

Wilcoxon-Mann-Whitney:
 1.6  1.7  1.8  1.9  2.1
 2.7  2.8  3.0  3.3  3.6
 3.6  4.0  4.3  4.8  5.3
 4.6  4.9  5.4  5.7  6.4
 5.2  5.6  6.2  6.5  6.9

Brunner-Munzel:
 5.0  5.0  5.2  5.0  5.2
 5.3  5.0  5.2  5.4  5.1
 5.2  5.2  5.2  5.3  5.3
 5.2  5.2  5.3  5.2  5.3
 5.1  5.2  5.3  5.1  5.2

Student t:
 1.5  3.1  5.3  7.4  8.1
 2.3  2.9  3.5  3.6  2.9
 3.2  3.4  3.3  2.9  2.6
 4.2  4.1  4.0  3.7  3.8
 5.1  5.1  5.2  5.4  6.0

Welch t:
 13.4  22.7  32.4  39.5  42.8
  8.9  12.5  15.8  18.1  18.5
  7.0   8.4   9.4   9.6   9.7
  6.1   6.5   6.8   6.6   6.4
  5.5   5.7   5.7   5.5   5.3

  7.931308 seconds (25.50 M allocations: 1.941 GiB, 4.61% gc time)


In [51]:
@time print_sim(; m=100, n=100, shifttype=:mean, list_skewness=list_skewness2)

Wilcoxon-Mann-Whitney:
 20.6  38.7  62.5  83.2  94.8
 12.5  21.4  35.7  56.7  78.9
  8.7  11.1  14.7  21.3  37.9
  6.6   7.0   6.5   5.8   5.1
  5.7   5.3   5.1   6.2  11.6

Brunner-Munzel:
 18.8  35.8  59.0  80.2  93.1
 12.0  20.7  34.7  55.0  76.9
  8.7  11.0  14.6  21.2  37.5
  6.7   7.0   6.5   5.7   5.0
  5.8   5.4   5.1   6.2  11.3

Student t:
 5.2  5.3  5.4  5.5  5.7
 5.0  5.2  5.1  5.2  5.3
 5.1  5.0  5.0  5.0  5.0
 4.9  5.1  5.1  4.9  4.8
 5.0  5.0  5.0  4.9  4.9

Welch t:
 5.2  5.2  5.3  5.5  5.7
 5.0  5.2  5.1  5.2  5.2
 5.1  5.0  5.0  5.0  5.0
 4.9  5.1  5.1  4.8  4.8
 5.0  5.0  5.0  4.9  4.9

 22.414538 seconds (25.99 M allocations: 2.080 GiB, 1.39% gc time)


In [52]:
@time print_sim(; m=100, n=100, shifttype=:median, list_skewness=list_skewness2)

Wilcoxon-Mann-Whitney:
 5.7  5.9  6.4   7.3   8.8
 5.4  5.4  5.2   5.4   7.7
 5.7  5.8  6.7   9.4  16.4
 6.1  6.6  8.4  13.2  21.5
 6.2  7.4  9.9  15.4  24.2

Brunner-Munzel:
 5.0  5.2  5.6   6.6   8.0
 5.2  5.2  5.2   5.4   7.8
 5.7  5.9  6.8   9.6  16.5
 6.2  6.7  8.5  13.2  21.0
 6.3  7.5  9.9  15.2  23.4

Student t:
 17.8  27.3  36.8  44.4  48.6
 13.5  17.8  21.5  23.4  23.4
 11.1  12.7  13.6  13.4  12.3
  9.6   9.8   9.5   8.7   7.5
  8.4   8.3   7.3   6.4   5.5

Welch t:
 17.7  27.1  36.6  44.2  48.3
 13.5  17.7  21.4  23.3  23.3
 11.1  12.7  13.6  13.4  12.2
  9.6   9.8   9.5   8.7   7.5
  8.4   8.3   7.3   6.4   5.5

 22.242460 seconds (27.08 M allocations: 2.389 GiB, 2.00% gc time)


In [53]:
@time print_sim(; m=100, n=100, shifttype=:tie, list_skewness=list_skewness2)

Wilcoxon-Mann-Whitney:
 5.8  5.9  5.7  5.6  5.5
 5.3  5.2  5.0  5.3  5.0
 5.0  5.1  5.1  5.0  5.1
 5.0  5.1  5.1  5.1  5.1
 5.0  5.0  5.1  5.2  5.2

Brunner-Munzel:
 5.1  5.2  5.0  5.1  5.1
 5.0  5.1  4.9  5.2  5.0
 5.0  5.1  5.1  5.1  5.1
 5.1  5.2  5.1  5.1  5.0
 5.0  5.1  5.0  5.1  5.0

Student t:
 17.4  30.7  44.4  54.2  58.0
 11.0  15.5  19.3  20.7  19.2
  8.0   8.7   8.8   7.9   6.8
  6.3   6.2   5.5   5.0   4.9
  5.6   5.1   4.9   5.4   6.3

Welch t:
 17.2  30.5  44.2  54.0  57.8
 10.9  15.4  19.2  20.6  19.1
  7.9   8.7   8.8   7.9   6.8
  6.3   6.2   5.5   5.0   4.9
  5.6   5.1   4.9   5.4   6.3

 22.155421 seconds (26.37 M allocations: 2.187 GiB, 1.70% gc time)
